In [2]:
# pip install faker psycopg2

In [8]:
import psycopg2
from psycopg2 import sql, extras
from faker import Faker
import random

# Initialize Faker and PostgreSQL connection details
fake = Faker()

# Database connection details
DB_HOST = 'localhost'
DB_NAME = 'ecom'
DB_USER = 'postgres'
DB_PASSWORD = 'password'
DB_PORT = '5432'

# Establish connection to PostgreSQL
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        port=DB_PORT
    )
    cursor = conn.cursor()
    print("Connected to the database successfully.")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    exit(1)

# Function to truncate tables before inserting new data
def truncate_tables():
    try:
        tables = ['order_items', 'orders', 'products', 'users', 'categories']
        for table in tables:
            query = sql.SQL("TRUNCATE TABLE {} RESTART IDENTITY CASCADE").format(sql.Identifier(table))
            cursor.execute(query)
        print("Tables truncated successfully.")
    except Exception as e:
        print(f"Error truncating tables: {e}")
        conn.rollback()

# Function to insert categories
def insert_categories():
    categories = ['Electronics', 'Clothing', 'Home & Kitchen', 'Sports', 'Beauty', 'Toys', 'Books']
    category_data = [(category,) for category in categories]
    try:
        query = sql.SQL("INSERT INTO categories (name) VALUES (%s)")
        extras.execute_batch(cursor, query, category_data)
        print("Categories inserted successfully.")
    except Exception as e:
        print(f"Error inserting categories: {e}")
        conn.rollback()

# Function to insert users with unique emails
def insert_users(num_users):
    user_data = []
    for _ in range(num_users):
        name = fake.name()
        email = fake.unique.email()  # Ensure unique emails
        address = fake.address().replace("\n", ", ")
        phone_number = fake.phone_number()[:15]  # Ensure phone number is within 15 characters
        user_data.append((name, email, address, phone_number))
    
    try:
        query = sql.SQL("INSERT INTO users (name, email, address, phone_number) VALUES (%s, %s, %s, %s)")
        extras.execute_batch(cursor, query, user_data)
        print(f"{num_users} users inserted successfully.")
    except Exception as e:
        print(f"Error inserting users: {e}")
        conn.rollback()

# Function to insert products
def insert_products(num_products):
    product_data = []
    for _ in range(num_products):
        name = fake.word().capitalize() + " " + fake.word().capitalize()
        description = fake.text(max_nb_chars=200)
        price = round(random.uniform(10, 500), 2)
        stock_quantity = random.randint(1, 100)
        category_id = random.randint(1, 7)  # Assuming there are 7 categories
        product_data.append((name, description, price, stock_quantity, category_id))
    
    try:
        query = sql.SQL("INSERT INTO products (name, description, price, stock_quantity, category_id) VALUES (%s, %s, %s, %s, %s)")
        extras.execute_batch(cursor, query, product_data)
        print(f"{num_products} products inserted successfully.")
    except Exception as e:
        print(f"Error inserting products: {e}")
        conn.rollback()

# Function to insert orders
def insert_orders(num_orders):
    order_data = []
    for _ in range(num_orders):
        user_id = random.randint(1, 1000)  # Assuming there are 1000 users
        order_date = fake.date_this_year()
        total_amount = round(random.uniform(10, 500), 2)
        order_status = random.choice(['Shipped', 'Pending'])
        order_data.append((user_id, order_date, total_amount, order_status))
    
    try:
        query = sql.SQL("INSERT INTO orders (user_id, order_date, total_amount, order_status) VALUES (%s, %s, %s, %s)")
        extras.execute_batch(cursor, query, order_data)
        print(f"{num_orders} orders inserted successfully.")
    except Exception as e:
        print(f"Error inserting orders: {e}")
        conn.rollback()

# Function to insert order items
def insert_order_items(num_order_items):
    order_item_data = []
    for _ in range(num_order_items):
        order_id = random.randint(1, 1000)  # Assuming there are 1000 orders
        product_id = random.randint(1, 500)  # Assuming there are 500 products
        quantity = random.randint(1, 3)
        price = round(random.uniform(10, 500), 2)
        order_item_data.append((order_id, product_id, quantity, price))
    
    try:
        query = sql.SQL("INSERT INTO order_items (order_id, product_id, quantity, price) VALUES (%s, %s, %s, %s)")
        extras.execute_batch(cursor, query, order_item_data)
        print(f"{num_order_items} order items inserted successfully.")
    except Exception as e:
        print(f"Error inserting order items: {e}")
        conn.rollback()

# Main insertion logic
try:
    # Truncate tables before inserting new data
    truncate_tables()

    # Insert categories first
    insert_categories()

    # Insert users, products, orders, and order items in the correct order
    insert_users(1000)  # Insert users first
    insert_products(500)
    insert_orders(1000)  # Insert orders after users
    insert_order_items(3000)  # Insert order items after orders

    # Commit the transaction
    conn.commit()
    print("All data inserted and committed successfully!")

except Exception as e:
    print(f"An error occurred: {e}")
    conn.rollback()

finally:
    # Close the cursor and the connection
    cursor.close()
    conn.close()
    print("Connection closed.")


Connected to the database successfully.
Tables truncated successfully.
Categories inserted successfully.
1000 users inserted successfully.
500 products inserted successfully.
1000 orders inserted successfully.
3000 order items inserted successfully.
All data inserted and committed successfully!
Connection closed.
